In [9]:
%pip install webdriver_manager

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import concurrent.futures
from webdriver_manager.chrome import ChromeDriverManager


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:

# Thiết lập Headless Chrome
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Chạy ẩn
# chrome_options.add_argument("--disable-gpu")  # Tắt GPU để giảm tải
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # Chặn tải ảnh
# chrome_options.add_argument("--disable-javascript")  # Chặn JavaScript

# Khởi tạo WebDriver
service = Service(ChromeDriverManager().install())  # Đường dẫn ChromeDriver
driver = webdriver.Chrome(service=service)

# URL trang web cần cào
URL = "https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures"  # Thay bằng URL thật của bảng dữ liệu

# Mở trang web
driver.get(URL)
time.sleep(3)  # Chờ trang tải

# Lấy nội dung trang bằng BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")

# Tìm tất cả các hàng trong bảng
rows = soup.select("tr[data-row]")

# Danh sách chứa thông tin trận đấu
match_data = []

for row in rows:
    try:
        match_info = {
            "Day": row.select_one("[data-stat='dayofweek']").text.strip(),
            "Date": row.select_one("[data-stat='date']").text.strip(),
            "Time": row.select_one("[data-stat='start_time']").text.strip(),
            "Home": row.select_one("[data-stat='home_team']").text.strip(),
            "xG_Home": row.select_one("[data-stat='home_xg']").text.strip(),
            "Score": row.select_one("[data-stat='score']").text.strip(),
            "xG_Away": row.select_one("[data-stat='away_xg']").text.strip(),
            "Away": row.select_one("[data-stat='away_team']").text.strip(),
            "Attendance": row.select_one("[data-stat='attendance']").text.strip(),
            "Venue": row.select_one("[data-stat='venue']").text.strip(),
            "Referee": row.select_one("[data-stat='referee']").text.strip(),
            "Match_Report_Link": row.select_one("[data-stat='match_report'] a")["href"] if row.select_one("[data-stat='match_report'] a") else None,
        }
        match_data.append(match_info)
    except Exception as e:
        print(f"Lỗi khi xử lý hàng: {e}")

# Chuyển dữ liệu thành DataFrame
df = pd.DataFrame(match_data)

# Lưu tạm DataFrame này
df.to_csv("match_info.csv", index=False)


driver.quit()


In [12]:
df = pd.read_csv(r'D:\Desktop\tap tanh hoc code\.vscode\DAP\match_info.csv')

In [17]:
df.dropna()

,Day,Date,Time,Home,xG_Home,Score,xG_Away,Away,Attendance,Venue,Referee,Match_Report_Link
0,Fri,2024-08-16,20:00 (02:00),Manchester Utd,2.4,1–0,0.4,Fulham,"73,297",Old Trafford,Robert Jones,/en/matches/cc5b4244/Manchester-United-Fulham-...
1,Sat,2024-08-17,12:30 (18:30),Ipswich Town,0.5,0–2,2.6,Liverpool,"30,014",Portman Road Stadium,Tim Robinson,/en/matches/a1d0d529/Ipswich-Town-Liverpool-Au...
2,Sat,2024-08-17,15:00 (21:00),Newcastle Utd,0.3,1–0,1.8,Southampton,"52,196",St James' Park,Craig Pawson,/en/matches/34557647/Newcastle-United-Southamp...
3,Sat,2024-08-17,15:00 (21:00),Nott'ham Forest,1.3,1–1,1.2,Bournemouth,"29,763",The City Ground,Michael Oliver,/en/matches/4efc72e4/Nottingham-Forest-Bournem...
4,Sat,2024-08-17,15:00 (21:00),Everton,0.5,0–3,1.4,Brighton,"39,217",Goodison Park,Simon Hooper,/en/matches/71618ace/Everton-Brighton-and-Hove...
...,...,...,...,...,...,...,...,...,...,...,...,...
294,Sat,2025-02-22,15:00 (22:00),Fulham,0.2,0–2,1.2,Crystal Palace,"26,777",Craven Cottage,Robert Jones,/en/matches/7289bcdf/Fulham-Crystal-Palace-Feb...
295,Sat,2025-02-22,15:00 (22:00),Ipswich Town,1.1,1–4,1.6,Tottenham,"30,003",Portman Road Stadium,Tim Robinson,/en/matches/7853cd1a/Ipswich-Town-Tottenham-Ho...
296,Sat,2025-02-22,15:00 (22:00),Bournemouth,0.6,0–1,2.0,Wolves,"11,206",Vitality Stadium,Michael Salisbury,/en/matches/ccdda6d3/Bournemouth-Wolverhampton...
297,Sat,2025-02-22,15:00 (22:00),Arsenal,1.2,0–1,1.2,West Ham,"60,262",Emirates Stadium,Craig Pawson,/en/matches/e511e91c/Arsenal-West-Ham-United-F...


In [21]:
finished_match_info = df.dropna()
# lưu lại
finished_match_info.to_csv('finished_matches_info.csv')


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Day                400 non-null    object
 1   Date               400 non-null    object
 2   Time               400 non-null    object
 3   Home               400 non-null    object
 4   xG_Home            279 non-null    object
 5   Score              279 non-null    object
 6   xG_Away            279 non-null    object
 7   Away               400 non-null    object
 8   Attendance         277 non-null    object
 9   Venue              400 non-null    object
 10  Referee            279 non-null    object
 11  Match_Report_Link  380 non-null    object
dtypes: object(12)
memory usage: 41.4+ KB
